# Julia Multi-Processing

[![ ](https://markdown-videos-api.jorgenkh.no/url?url=https%3A%2F%2Fyoutu.be%2FuKdpY6Je6Oc%3Flist%3DPLhQ2JMBcfAsjQzwp2j97uZjNOMi7Ed4CG)](https://youtu.be/uKdpY6Je6Oc?list=PLhQ2JMBcfAsjQzwp2j97uZjNOMi7Ed4CG)

### 👉 single-process vs multi-process

In [ ]:
using Distributed

In [ ]:
addprocs(10)

In [ ]:
procs()

In [ ]:
# single-process
# Re-run this cell several times

@time for i in 2:nprocs()
    sleep(1)
end

In [ ]:
# multi-process
# Re-run this cell several times

@time @sync for i in 2:nprocs()
    @spawnat i sleep(1)
end

🚩 multi-processing is 10 times faster but uses more memory

#### another example

In [ ]:
# single-process
# Re-run this cell several times

@time for i in 2:nprocs()
    println("Hello, World!")
end

In [ ]:
# multi-process
# Re-run this cell several times

@time @sync for i in 2:nprocs()
    @spawnat i println("Hello, World!")
end

🚩 in this example, multi-processing is slower than single-processing !!!

In [ ]:
# removing (killing) the created procs 

rmprocs(procs()[2:end], waitfor=0)

In [ ]:
procs()

### <font color="red"> !!! RESTART THE KERNEL NOW !!! </font>
------------------------

## 🔖 cheatsheet

In [ ]:
import Pkg

In [ ]:
# make a new directory

mkdir("08x04")

In [ ]:
# activate the environment

Pkg.activate("08x04") 

In [ ]:
# change the working directory

cd("08x04")

In [ ]:
# check the current directory

pwd()

In [ ]:
# add packages to the new environment

Pkg.add("Plots")

In [ ]:
using Distributed

In [ ]:
# add 5 workers

addprocs(5)

In [ ]:
nprocs()

In [ ]:
procs()

In [ ]:
workers()

In [ ]:
# remove (kill) a proccess (worker)

rmprocs(6)

In [ ]:
procs()

In [ ]:
workers()

In [ ]:
# how to track workers (optional)

w = workers()

In [ ]:
w[1]

In [ ]:
# assign a work to a worker

ans1 = @spawnat w[1] rand(3, 4)

In [ ]:
# fetch a result from a worker

fetch(ans1)

In [ ]:
# assign a task to any available worker

ans2 = @spawnat :any sum(1:100)

In [ ]:
fetch(ans2)

#### 💡 combine @spawnat and fetch()

In [ ]:
ans3 = @spawnat w[2] 3.14

In [ ]:
fetch(ans3)

In [ ]:
ans4 = @spawnat w[3] 1 + fetch(ans3)

In [ ]:
fetch(ans4)

In [ ]:
# add standard library to all workers

@everywhere using Statistics

In [ ]:
ans5 = @spawnat w[4] mean(rand(100))

In [ ]:
fetch(ans5)

In [ ]:
# add standard library to a specific worker

@everywhere w[1] using LinearAlgebra

In [ ]:
ans6 = @spawnat w[1] axpy!(fetch(ans3), ones(10), ones(10))

In [ ]:
fetch(ans6)

In [ ]:
ans7 = @spawnat w[2] axpy!(fetch(ans3), ones(10), ones(10))

In [ ]:
fetch(ans7)

##### 🚩🚩🚩 only w[1] can use LinearAlgebra functions !!!

#### 💡 use SharedArrays

In [ ]:
using SharedArrays

In [ ]:
data = SharedArray(rand(10))

#### 💡 add external package to all workers

In [ ]:
@everywhere import Pkg

In [ ]:
# activate the environment for all the workers

@everywhere Pkg.activate(".")

In [ ]:
@everywhere using Plots

In [ ]:
ans8 = @spawnat w[3] plot(data)

In [ ]:
fetch(ans8)

#### 💡 how to use a module

In [ ]:
# create a module file

write("ToyModule.jl",
"module ToyModule

export myfunc

function myfunc()
    println(\"Hello, World!\")
    return 2.718
end

end"
    );

In [ ]:
@everywhere include("ToyModule.jl")

In [ ]:
using .ToyModule

In [ ]:
ans9 = @spawnat w[4] ToyModule.myfunc()

In [ ]:
fetch(ans9)

In [ ]:
# removing (killing) the created procs 

rmprocs(procs()[2:end], waitfor=0)

In [ ]:
procs()

### <font color="red"> !!! RESTART THE KERNEL NOW !!! </font>
------------------------

## ⚙️ demo project

In [ ]:
# change directory

cd("08x04")

In [ ]:
# check the current directory

pwd()

In [ ]:
import Pkg

In [ ]:
# activate the environment

Pkg.activate(".")

In [ ]:
# create a module file

write("MyFunctions.jl",
"module MyFunctions
    
export task1, task2, task3
    
# tasks
    
function task1(data)
    data * data
end
    
function task2(data)
    data / data
end
    
function task3(data)
    data \\ data
end

# print message

println(\"MyFunctions loaded\")
    
end"
    );

In [ ]:
# add workers

using Distributed

addprocs(4)
w = workers()

In [ ]:
# generate random data

using SharedArrays

n = 1_000
data = SharedMatrix(rand(n, n))
x = data[: ,1]
y = data[:, 2];

In [ ]:
# load module

@everywhere include("MyFunctions.jl")

In [ ]:
using .MyFunctions

In [ ]:
# load package

@everywhere import Pkg

In [ ]:
# activate environment for all workers

@everywhere Pkg.activate(".")

In [ ]:
@everywhere using Plots

In [ ]:
# launch tasks

t1 = @spawnat w[1] MyFunctions.task1(data)
t2 = @spawnat w[2] MyFunctions.task2(data)
t3 = @spawnat w[3] MyFunctions.task3(data)

In [ ]:
t4 = @spawnat w[4] scatter(
    (x,y),
    legend = false,
    aspect_ratio = 1
)

In [ ]:
# fetch results

fetch(t1)
fetch(t2)
fetch(t3)
fetch(t4)

In [ ]:
# removing (killing) the created procs 

rmprocs(procs()[2:end], waitfor=0)

In [ ]:
procs()